In [5]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from dotenv import load_dotenv
import os
from pprint import pprint
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET=os.getenv("CLIENT_SECRET")

spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id = CLIENT_ID, client_secret = CLIENT_SECRET))

Going to aim to get a list of track IDs from the playlist endpoint. The only way I can think to build the dataset is just by bruteforcing the audio_analysis endpoint.

<iframe src="https://open.spotify.com/embed/playlist/37i9dQZF1DWYz61oV0Yc4H" width="100%" height="380" frameBorder="0" allowtransparency="true" allow="encrypted-media"></iframe>

In [8]:
results = spotify.playlist("spotify:playlist:37i9dQZF1DWYz61oV0Yc4H")
print(results.keys())

dict_keys(['collaborative', 'description', 'external_urls', 'followers', 'href', 'id', 'images', 'name', 'owner', 'primary_color', 'public', 'snapshot_id', 'tracks', 'type', 'uri'])


In [ ]:
tracks = results["tracks"]["items"]
track = tracks [0]
track.keys()
track["track"]["id"]

id_list = [track["track"]["id"] for track in tracks]
id_list

In [6]:
def get_track_ids(playlist_id):
    results = spotify.playlist(f"spotify:playlist:{playlist_id}")
    tracks = results["tracks"]["items"]
    return [track["track"]["id"] for track in tracks]

get_track_ids("37i9dQZF1DWYz61oV0Yc4H")


['1NGPZKzplieiPc5g6lAJ49',
 '0sxx8bEWlTBmbw0X1Abgc3',
 '4jFLw7QqWlv3lZr980HyYW',
 '3r1qdSsvJEwiMKHeCWapMM',
 '6dpLVFP6V0TZ17zmgRTf6q',
 '6EX2dd9xt2fR9XcKwfN1Ld',
 '63w36H13PifYssKKvV3W8y',
 '1P7tiVl5hgemZuH8oNyP6C',
 '5JWPUEov2wlX7c0jhYZpeB',
 '49UDOG8DoBajXTJSTqfRMg',
 '0OM7XnLJnT441emHL5upvW',
 '6V8IY1khSjXlgvG6mbIyXQ',
 '5X8FVIy9iaXoDEiIlCjcmp',
 '3DXEbjIftmPeBj638CxyJD',
 '3MMLO6TUL07tze9IXud0Vc',
 '4JOkWaNiPjycCHgYuiJjzy',
 '2xLNWLmeNt8IUv0zTUMqGF',
 '778JDHI6qfgQ1HBEKP1FZu',
 '5gzm0zFeHjYfteQu7x2uus',
 '3YrT4nWBMF1eEHYc2fNOht',
 '5lS6b95oJTkSxhg2BEbd4f',
 '15gL4n95OG5KEQLCMXPQzp',
 '58yHjcmgpNmNDNZUYtju6e',
 '6gESvyTKFOm65GCJibT8kI',
 '3aMyQSDW0DJ7LTOkrd29hU',
 '08WlCqv58b7ivVAHcOygTp',
 '7aoQKm7L9RGg5VxYitUkEG',
 '0KGc8uyIXJdZjyMe8Jihdk',
 '2vjo6HuhOURw0wXsb0EVCa',
 '3EuQ0cCmo81wMxM2wTn5Yh',
 '3OZfyZHECqT3q48x5ct2yh',
 '3kcX1eNaWhs3uqgwpRWgID',
 '2kNtYu5hPVx2oZUHjbwyTa',
 '1oblFfkg4d9hWVMQU1c7NW',
 '1oUnzYmReVbOPaFK7e06fk',
 '0g2PEMHME7QjuXyC2n1iR7',
 '5MzNoRzcRrXH6kDtwnEAJD',
 

In [29]:
def unnest_lists(df : pd.DataFrame):
    for col, dtype in df.dtypes.to_dict().items():
        if dtype == object and hasattr(df.loc[0,col],"__iter__"):
            temp = df[col].apply(pd.Series)
            new_names = [f"{col}_{column}" for column in temp.columns]
            temp.rename(columns=dict(zip(temp.columns,new_names)),inplace = True)
            df= pd.concat([df,temp],axis=1)
            df.drop(columns=col,inplace=True)
    return df

            


def create_sample(track_id):
    results = spotify.audio_analysis(f"spotify:track:{track_id}")
    df=pd.json_normalize(results["segments"])
    df = unnest_lists(df)
 
    return df

create_sample("1NGPZKzplieiPc5g6lAJ49")

    

,start,duration,confidence,loudness_start,loudness_max_time,loudness_max,loudness_end,pitches_0,pitches_1,pitches_2,...,timbre_2,timbre_3,timbre_4,timbre_5,timbre_6,timbre_7,timbre_8,timbre_9,timbre_10,timbre_11
0,0.00000,0.37220,1.000,-48.496,0.12681,-12.417,0.0,0.127,0.164,0.143,...,-87.023,-65.421,17.423,69.830,16.590,-18.424,-2.835,38.497,55.099,1.460
1,0.37220,0.28621,0.580,-22.486,0.00858,-13.915,0.0,0.042,0.092,0.003,...,-140.188,47.534,27.989,-86.531,-14.547,-102.794,-11.281,48.107,-32.784,19.484
2,0.65841,0.42336,1.000,-28.819,0.08816,-3.037,0.0,0.045,0.040,0.051,...,-55.236,-123.233,27.741,22.460,23.532,-4.947,4.520,-21.835,-34.083,11.718
3,1.08177,0.27633,0.609,-24.663,0.00829,-16.506,0.0,0.091,0.115,0.127,...,-127.389,29.478,19.778,-76.488,10.980,-82.504,-5.367,30.322,-18.353,20.122
4,1.35810,0.39320,0.810,-29.147,0.14189,-13.129,0.0,0.093,0.334,0.665,...,-68.824,-83.992,-13.621,-14.006,7.838,-15.925,-2.227,-10.826,-39.420,8.366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
822,221.55882,0.35288,0.963,-46.013,0.04279,-33.254,0.0,0.085,0.639,0.061,...,-63.401,33.135,61.252,16.917,-8.244,-11.768,2.473,-21.609,2.111,-16.767
823,221.91170,0.16966,0.414,-47.389,0.06385,-43.318,0.0,0.904,0.857,0.211,...,-29.771,-34.522,44.316,-29.221,0.357,16.266,-14.303,-3.372,-16.324,-21.365
824,222.08136,0.28685,0.275,-48.389,0.07583,-45.011,0.0,0.250,0.994,1.000,...,-34.694,-39.279,29.607,-45.008,38.116,14.098,-23.590,6.864,-8.090,-11.987
825,222.36821,0.61878,0.033,-46.661,0.02204,-44.194,0.0,0.046,0.074,0.051,...,-66.430,0.545,33.354,-54.565,1.692,37.169,-8.354,-8.690,-9.430,-7.005
